In [ ]:
import pandas as pd
import os
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

Selected classes: ['Dimorphodon', 'Dilophosaurus', 'Triceratops', 'Parasaurolophus']
Found 537 validated image filenames belonging to 4 classes.
Found 134 validated image filenames belonging to 4 classes.


In [7]:
# Define the base directory where the 'archive-2' folder is located
base_dir = "dinosaur_dataset"

# Define the batch size and target image size for the model
batch_size = 32
target_image_size = (224, 224)

# Set up the ImageDataGenerators with a split for training and testing
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2,
)

# Create the train generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=target_image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

# Create the test generator
test_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=target_image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)
# Load the base model
base_model = tf.keras.applications.ResNet152V2(
    weights="imagenet", include_top=False, input_shape=target_image_size + (3,)
)

# Start defining the custom classification head
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Add a dropout layer before the dense layer
# Let's say you want to set 50% of the inputs to zero
x = tf.keras.layers.Dropout(0.5)(x)  # Adjust the dropout rate as needed

x = tf.keras.layers.Dense(512, activation="relu")(x)

# Continue with the final dense layer for classification
predictions = tf.keras.layers.Dense(15, activation="softmax")(x)

# Combine the base model with the custom classification head
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Now proceed with compilation and training as before

print("Number of layers in the base model: ", len(base_model.layers))

# Freeze the base model layers
for layer in base_model.layers[:530]:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=test_generator,
    validation_steps=len(test_generator),
)

Found 1964 images belonging to 15 classes.
Found 484 images belonging to 15 classes.
234545216/234545216 [==============================] - 12s 0us/step
Number of layers in the base model:  564
Epoch 1/15
62/62 [==============================] - 67s 798ms/step - loss: 1.7463 - accuracy: 0.4557 - val_loss: 39.0178 - val_accuracy: 0.2459
Epoch 2/15
62/62 [==============================] - 47s 755ms/step - loss: 1.0801 - accuracy: 0.6522 - val_loss: 3.0672 - val_accuracy: 0.5455
Epoch 3/15
62/62 [==============================] - 47s 765ms/step - loss: 0.8461 - accuracy: 0.7271 - val_loss: 3.0880 - val_accuracy: 0.5455
Epoch 4/15
62/62 [==============================] - 48s 771ms/step - loss: 0.6844 - accuracy: 0.7841 - val_loss: 1.4656 - val_accuracy: 0.6570
Epoch 5/15
62/62 [==============================] - 47s 763ms/step - loss: 0.6619 - accuracy: 0.8024 - val_loss: 1.8444 - val_accuracy: 0.5847
Epoch 6/15
62/62 [==============================] - 47s 755ms/step - loss: 0.4797 - accura

In [14]:
model.save('dinosaur_classifier.keras')